<a href="https://colab.research.google.com/github/Dansoeun/AICOSS/blob/ChatBot/TextPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import pandas as pd

In [2]:
def counting(path):
    cnt = 0
    data_dir = path

    for path in os.listdir(data_dir):
        if os.path.isfile(os.path.join(data_dir, path)):
            cnt += 1

    print(f'데이터 개수 = {cnt}')
    return cnt

# CSV 데이터 처리

In [3]:
target_path = pd.read_csv("/content/drive/MyDrive/AICOSS/knu_dataset_csv.csv", encoding = "UTF-8")

In [4]:
target_path

,Question,Answer
0,"치대 학생인데, 장학금을 받고싶어.",https://dent.knu.ac.kr/
1,의학과 교육과정을 알려줘,https://med.knu.ac.kr/pages/sub.htm?nav_code=k...
2,컴학 공지사항 알려줘,https://cse.knu.ac.kr/bbs/board.php?bo_table=s...
3,AIcoss 학생인데 gpu 서버 사용하고 싶어,https://aicoss.knu.ac.kr/gpu/list.do
4,인컴 교육과정 알려줘,https://cse.knu.ac.kr/sub3_3_b.php
...,...,...
190,봉사활동을 가고 싶은데 어캐해야할까,https://volunteer.knu.ac.kr/
191,수강 신청 하고 싶어,https://sugang.knu.ac.kr/login.knu
192,비교과 사이트가 궁금해,https://knucube.knu.ac.kr/knucube_index.jsp
193,예대에 대해 알고 싶어,https://mvarts.knu.ac.kr/


In [5]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 26.0 MB/s eta 0:00:00


# 텍스트 전처리 및 테스트

In [6]:
from konlpy.tag import Komoran

class Preprocess:
    def __init__(self, userdic=None):
        self.komoran = Komoran(userdic=userdic)

        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC',
            'SF', 'SP', 'SS', 'SE', 'SO',
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            'XSN', 'XSV', 'XSA'
        ]

    # pos 메서드 정의
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # get_keywords 메서드 정의
    def get_keywords(self, pos, without_tag=False):
        word_list = []
        for p in pos:
            if p[1] not in self.exclusion_tags:
                word_list.append(p if not without_tag else p[0])
        return word_list

# 코드 실행
sent = "컴학 공지사항을 알려줘"

p = Preprocess(userdic="/content/drive/MyDrive/AICOSS/knu_dataset_csv.csv.tsv")

# 형태소 분석 실행
pos = p.pos(sent)

# 태그 포함 키워드 추출
ret = p.get_keywords(pos, without_tag=False)
print(ret)

# 태그 제외 키워드 추출
ret = p.get_keywords(pos, without_tag=True)
print(ret)


[('컴', 'NNG'), ('학', 'NNG'), ('공지', 'NNP'), ('사항', 'NNG'), ('알리', 'VV'), ('주', 'VX')]
['컴', '학', '공지', '사항', '알리', '주']


# 단어 사전 구축

In [7]:
import pandas as pd

# CSV 파일을 불러오고 첫 몇 줄과 열 이름을 확인합니다.
topic = pd.read_csv("/content/drive/MyDrive/AICOSS/knu_dataset_csv.csv")

# 열 이름 출력
print(topic.columns)

Index(['Question', 'Answer'], dtype='object')


In [8]:
import pandas as pd
from konlpy.tag import Komoran
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

topic = pd.read_csv("/content/drive/MyDrive/AICOSS/knu_dataset_csv.csv")

topic.dropna(inplace=True)

text = list(topic['Question'])

corpus_data = text

p = Preprocess()
dict = []
for c in corpus_data:
  pos = p.pos(c)
  for k in pos:
    dict.append(k[0])

tokenizer = Tokenizer(oov_token='OOV', num_words=100000)
tokenizer.fit_on_texts(dict)
word_index = tokenizer.word_index
print(f"사전에 포함된 단어 수: {len(word_index)}")

with open("chatbot_dict.bin", "wb") as f:
    pickle.dump(word_index, f)

사전에 포함된 단어 수: 439


# 단어 사전 테스트

In [9]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer

f = open("/content/chatbot_dict.bin", "rb")
word_index = pickle.load(f)
f.close()

sent = "안녕하세요. 테스트 코드입니다. 단어사전 만들기 성공!"

p = Preprocess(userdic="/content/drive/MyDrive/AICOSS/knu_dataset_csv.csv.tsv")

pos = p.pos(sent)

keywords = p.get_keywords(pos, without_tag=True)
for word in keywords:
  try:
    print(word, word_index[word])
  except KeyError:
    print(word, word_index['OOV'])

안녕하세요 1
테스트 1
코드 1
이 11
단어 1
사전 1
만들 401
성공 1


In [10]:
from konlpy.tag import Komoran
import pickle

class Preprocess:
  def __init__(self, word2index_dic="/content/chatbot_dict.bin", userdic="user_dic.tsv"):
    if(word2index_dic != ''):
      f = open(word2index_dic, "rb")
      self.word_index = pickle.load(f)
      f.close()
      print("success load word_dic")
    else:
      self.word_index = None
      print("fail load word_dic")

    self.komoran = Komoran(userdic=userdic)

    self.exclusion_tags = [
        'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
        'JX', 'JC',
        'SF' 'SP', 'SS', 'SE', 'SO',
        'EP', 'EF', 'EC', 'ETN', 'ETM',
        'XSN', 'XSV', 'XSA'
    ]

  def pos(self, sentence):
    return self.komoran.pos(sentence)

  def get_keywords(self, pos, without_tag=False):
    f = lambda x: x in self.exclution_tags
    word_list = []
    for p in pos:
      if f(p[1]) is False:
        word_list.append(p if without_tag is False else p[0])
    return word_list

  def get_wordidx_sequence(self, keywords):
    if self.word_index is None:
      return []
    w2i = []
    for word in keywords:
      try:
        w2i.append(self.word_index[word])
      except KeyError:
        w2i.append(self.word_index['OOV'])
    return w2i